In [17]:
!pip install gym_super_mario_bros==7.3.0 nes_py

In [18]:
import  gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [19]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


In [20]:
!pip install stable-baselines3[extra]

In [21]:
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from matplotlib import pyplot as plt

In [22]:
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
env = GrayScaleObservation(env, keep_dim=True)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [23]:
state = env.reset()

In [24]:
import os
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback

In [25]:
class TrainAndLoggingCallback(BaseCallback):
    
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
        
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
    
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
        return True

In [26]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [27]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [14]:
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.0001, n_steps=512)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [15]:
model.learn(total_timesteps=1000000, callback=callback)

Logging to ./logs/PPO_6


C:\Users\Xx-Th\AppData\Local\Programs\Python\Python310\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


----------------------------
| time/              |     |
|    fps             | 62  |
|    iterations      | 1   |
|    time_elapsed    | 8   |
|    total_timesteps | 512 |
----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 15          |
|    iterations           | 2           |
|    time_elapsed         | 67          |
|    total_timesteps      | 1024        |
| train/                  |             |
|    approx_kl            | 0.020762473 |
|    clip_fraction        | 0.209       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.93       |
|    explained_variance   | 0.00181     |
|    learning_rate        | 0.0001      |
|    loss                 | 10.5        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0125     |
|    value_loss           | 77.4        |
-----------------------------------------
----------------------------------------


------------------------------------------
| time/                   |              |
|    fps                  | 8            |
|    iterations           | 13           |
|    time_elapsed         | 752          |
|    total_timesteps      | 6656         |
| train/                  |              |
|    approx_kl            | 0.0155429505 |
|    clip_fraction        | 0.115        |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.82        |
|    explained_variance   | 0.162        |
|    learning_rate        | 0.0001       |
|    loss                 | 0.0518       |
|    n_updates            | 120          |
|    policy_gradient_loss | -0.00646     |
|    value_loss           | 0.308        |
------------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 8           |
|    iterations           | 14          |
|    time_elapsed         | 815         |
|    total_times

------------------------------------------
| time/                   |              |
|    fps                  | 8            |
|    iterations           | 24           |
|    time_elapsed         | 1427         |
|    total_timesteps      | 12288        |
| train/                  |              |
|    approx_kl            | 0.0113665275 |
|    clip_fraction        | 0.127        |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.7         |
|    explained_variance   | 0.0708       |
|    learning_rate        | 0.0001       |
|    loss                 | 0.235        |
|    n_updates            | 230          |
|    policy_gradient_loss | -0.00495     |
|    value_loss           | 0.457        |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 8            |
|    iterations           | 25           |
|    time_elapsed         | 1492         |
|    total_

...

----------------------------------------
| time/                   |            |
|    fps                  | 7          |
|    iterations           | 1047       |
|    time_elapsed         | 68551      |
|    total_timesteps      | 536064     |
| train/                  |            |
|    approx_kl            | 0.07429621 |
|    clip_fraction        | 0.334      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.372     |
|    explained_variance   | 0.653      |
|    learning_rate        | 0.0001     |
|    loss                 | 46         |
|    n_updates            | 10460      |
|    policy_gradient_loss | 0.0351     |
|    value_loss           | 151        |
----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 7           |
|    iterations           | 1048        |
|    time_elapsed         | 68626       |
|    total_timesteps      | 536576      |
| train/  

------------------------------------------
| time/                   |              |
|    fps                  | 7            |
|    iterations           | 1058         |
|    time_elapsed         | 69419        |
|    total_timesteps      | 541696       |
| train/                  |              |
|    approx_kl            | 0.0035361066 |
|    clip_fraction        | 0.077        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.178       |
|    explained_variance   | 0.862        |
|    learning_rate        | 0.0001       |
|    loss                 | 9.49         |
|    n_updates            | 10570        |
|    policy_gradient_loss | 0.00322      |
|    value_loss           | 80.4         |
------------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 7           |
|    iterations           | 1059        |
|    time_elapsed         | 69499       |
|    total_times

----------------------------------------
| time/                   |            |
|    fps                  | 7          |
|    iterations           | 1069       |
|    time_elapsed         | 70261      |
|    total_timesteps      | 547328     |
| train/                  |            |
|    approx_kl            | 0.00621188 |
|    clip_fraction        | 0.0875     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.351     |
|    explained_variance   | 0.869      |
|    learning_rate        | 0.0001     |
|    loss                 | 13.5       |
|    n_updates            | 10680      |
|    policy_gradient_loss | -0.0058    |
|    value_loss           | 61.3       |
----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 7           |
|    iterations           | 1070        |
|    time_elapsed         | 70338       |
|    total_timesteps      | 547840      |
| train/  

-----------------------------------------
| time/                   |             |
|    fps                  | 7           |
|    iterations           | 1080        |
|    time_elapsed         | 71065       |
|    total_timesteps      | 552960      |
| train/                  |             |
|    approx_kl            | 0.011893289 |
|    clip_fraction        | 0.0779      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.218      |
|    explained_variance   | 0.617       |
|    learning_rate        | 0.0001      |
|    loss                 | 10.1        |
|    n_updates            | 10790       |
|    policy_gradient_loss | 0.00419     |
|    value_loss           | 231         |
-----------------------------------------


KeyboardInterrupt: 

In [28]:
model = PPO.load('./train/best_model_500000')

In [29]:
state = env.reset()
while True:
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()

KeyboardInterrupt: 

In [30]:
env.close()